<a href="https://colab.research.google.com/github/priyankasT16/Hackerearth-competitions/blob/master/Melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split,GroupKFold,StratifiedKFold,KFold
import random
import gc
from keras.utils import to_categorical
import os
import tensorflow as tf
from keras import backend as K
from keras.layers import Dense
from keras.models import Sequential
#import efficientnet.keras  as enf
from keras.callbacks import ReduceLROnPlateau
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda,GlobalAveragePooling2D
from math import ceil
import tensorflow as tf






Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
seed_value= 0
def seed_everything(seed):
  os.environ['PYTHONHASHSEED']=str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  # for later versions: 
  tf.compat.v1.set_random_seed(seed)
  return('Set seed {}'.format(seed))

seed_everything(seed_value)


'Set seed 0'

In [0]:
train=pd.read_csv('/content/drive/My Drive/kaggle_melanoma/train.csv')
test=pd.read_csv('/content/drive/My Drive/kaggle_melanoma/test.csv')
#submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

In [0]:
train=pd.DataFrame(train)

In [0]:
np.random.shuffle(train.values)

In [0]:
gkf=StratifiedKFold(n_splits=5)
train.loc[:,'Fold']=0
for fold_no,(train_index,val_index) in  enumerate(gkf.split(X=train.index,y=train['target'])):
    train.loc[train.iloc[val_index].index, 'Fold'] = fold_no

In [0]:
Y=train.target
#Y=to_categorical(Y)
#Y.shape


In [0]:
x_train=np.load('/content/drive/My Drive/kaggle_melanoma/x_train_32.npy')/255.0
test_file=np.load('/content/drive/My Drive/kaggle_melanoma/x_test_32.npy')/255.0

    

In [9]:
import gc
gc.collect()

39

In [0]:
#x_train=x_train/255.0
img_size=32

In [0]:
#x_tr,x_ts,y_tr,y_ts=train_test_split(x_train,Y,stratify=Y,shuffle=True,test_size=0.2)

In [0]:
#!pip install '../input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl'
!pip install -q efficientnet
import efficientnet.keras as efn

In [0]:
fold=1
train_1_index=train[train['Fold']!=fold].index
val_1_index=train[train['Fold']==fold].index

In [0]:
train_1_data=x_train[train_1_index]
val_1_data=x_train[val_1_index]
train_1_label=Y[train_1_index]
val_1_label=Y[val_1_index]

In [0]:
#train_1_index,Y[33117]

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  

datagen.fit(train_1_data)

In [0]:
'''
import pickel as pickle
x_pk=open('xpickle.pickle','wb')
pickle.dump(X,x_pk)
x_pk.close()
y_pk=open('ypickle.pikle','wb')
pickle.dump(Y,y_pk)
y_pk.close()
x_in=open('xpickle.pickle','rb')
train_x=pickle.load(x_in)
y_in=open('ypickle.pickle','rb')
train_y=pickle.load(y_in)
'''

In [0]:
import tensorflow as tf
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    #import tensorflow
    #gm_exp = tf.Variable(3.0, dtype = tf.float32)
    #import tensorflow as tf
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

In [0]:
dense_1=256
dense_2=128

In [0]:
def create_model(input_shape):
    # Input Layer
    input = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = efn.EfficientNetB7(weights = 'noisy-student', include_top = False, input_tensor = input, pooling = None, 
                                 classes = None)
    
    # UnFreeze all layers
    for layer in x_model.layers:
        layer.trainable = True
    
    # GeM
    lambda_layer = GlobalAveragePooling2D()
    #lambda_layer.trainable_weights.extend([gm_exp])
    x = lambda_layer(x_model.output)
     # multi output
    #x=Dense(dense_1,activation='relu')(x)
    #x=Dense(dense_2,activation='relu')(x)
    classes= Dense(1, activation = 'sigmoid', name = 'class')(x)
    

    # model
    model = Model(inputs = x_model.input, outputs = [classes])
    return model

In [17]:
#gc.collect()
img_size

32

In [0]:
HEIGHT_NEW=WIDTH_NEW=img_size
CHANNELS=3

model1 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))

In [0]:
model_name='B7X32n_1_64.h5'
#model1.save(model_name)

In [0]:
from tensorflow.keras import backend as K
import dill
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [20]:
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',factor=0.2,patience=2,cooldown=2,min_lr=0.00001,verbose=1)
callbacks = [reduce_learning_rate]
model1.compile( optimizer='adam',loss=[binary_focal_loss(alpha=.25, gamma=2)],metrics=[tf.keras.metrics.AUC()])
model1.fit_generator(datagen.flow(train_1_data,train_1_label, batch_size=64),
                     validation_data=(val_1_data,val_1_label),epochs=50,callbacks=callbacks,workers=-1,use_multiprocessing=True)
'''
score=[]
accuracy=[]
models=[]
kfolds=StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
for tr,ts in kfolds.split(x_train,Y):
  datagen.fit(x_train[tr])
  model3.fit_generator(datagen.flow(x_train[tr],Y[tr], batch_size=32),epochs=10,callbacks=callbacks)
  #datagen.fit(x_train[ts])
  res=model3.evaluate(x_train[ts],Y[ts])
  accuracy.append(res[1])
  roc=roc_auc_score(Y[ts],model3.predict(x_train[ts]))
  scores.append(roc)
  models.append(model3)

'''
#model3.fit(x_tr,y_tr,batch_size=32,epochs=50,callbacks=callbacks)
model1.save(model_name)

Epoch 1/50
415/415 [==============================] - 279s 673ms/step - loss: 0.9410 - auc_1: 0.5583 - val_loss: 0.0485 - val_auc_1: 0.6002
Epoch 2/50
415/415 [==============================] - 185s 445ms/step - loss: 0.5346 - auc_1: 0.6364 - val_loss: 0.0441 - val_auc_1: 0.6530
Epoch 3/50
415/415 [==============================] - 184s 444ms/step - loss: 0.5319 - auc_1: 0.6691 - val_loss: 0.0390 - val_auc_1: 0.6798
Epoch 4/50
415/415 [==============================] - 182s 438ms/step - loss: 0.5353 - auc_1: 0.6906 - val_loss: 0.0464 - val_auc_1: 0.6935
Epoch 5/50
415/415 [==============================] - 180s 435ms/step - loss: 0.5411 - auc_1: 0.6917 - val_loss: 0.0417 - val_auc_1: 0.6947

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/50
415/415 [==============================] - 181s 436ms/step - loss: 0.4864 - auc_1: 0.7034 - val_loss: 0.0365 - val_auc_1: 0.7117
Epoch 7/50
415/415 [==============================] - 181s 436ms/step - loss: 

In [0]:
#model_name='B7X32n_1_64.h5'
model1.save(model_name)

In [0]:
model=[model1]
def comare_model(models):
    output=[]
    names=['effecientnet','vgg16','resnet']
    for mod in models:
        res=mod.evaluate(x_ts,y_ts)
        roc=roc_auc_score(y_ts,mod.predict(x_ts))
        output.append((res[0],res[1],roc))
    output=pd.DataFrame(output,index=names,columns=['loss','accuracy','roc_score'])
    return(output)
result=comare_model(model)
result.to_csv(model_name+str('.csv'))
result
        

NameError: ignored

In [0]:
model1.save('B7X32_noisy1_0.835.h5')


In [0]:
from keras.models import load_model
import tensorflow as tf
model3=load_model('/kaggle/input/model-first/32.h5',custom_objects={'tf': tf,'gm_exp':gm_exp})

In [0]:
from datetime import datetime
import pytz
tz_ist=pytz.timezone('Asia/Kolkata')
dtnow=datetime.now(tz_ist)
import time
timestr = dtnow.strftime("%Y%m%d-%H%M")
#print timestr
#op
#inverse={'0':'Food', '1':'misc', '2':'Attire', '3':'Decorationandsignage'}
#inverse={'0':'manipuri', '1':'bharatanatyam', '2':'odissi', '3':'kathakali', '4':'kathak','5':'sattriya','6':'kuchipudi',
 #        '7':'mohiniyattam'}
labels = model1.predict(test_file)
#print(labels[:4])
l1 = [i[0]  for i in labels]
#class_label = [inverse[str(x)] for x in label]
#print(class_label[:3])
submission = pd.DataFrame({ 'image_name': test.image_name, 'target': l1 })
submission.head(10)
submission.to_csv('submission{}.csv'.format(timestr), index=False)

In [0]:
l1=[i[0] for i  in op]

In [0]:
l1

In [0]:
test_file.shape,len(label),print(set(label))

In [0]:
Y.unique(),y_tr.unique(),y_ts.unique()

In [0]:
op=model3.predict(test_file)

In [0]:
op.shape

In [0]:
oop= [1  if i>0.5 else 0 for i in op]

In [0]:
set_op=set(oop)
set_op
len(oop)

In [0]:
img_size=32
def preprocess_img(path,dsize=img_size):
    i=Image.open(path)
    i=i.resize((dsize,)*2,resample=Image.LANCZOS)
    return i

n=test.shape[0]
ts_df=np.empty((n,img_size,img_size,3),dtype=np.uint8)
for i,img in enumerate(tqdm(test.image_name)):
    ts_df[i,:,:,:]=preprocess_img(f'../input/siim-isic-melanoma-classification/jpeg/test/{img}.jpg')